# Cyclostationary Analysis

## Headers

In [1]:
# Import functions and libraries
import numpy as np
import threading,time
import sys
from cyclostationary import scd_fam

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy.random import randn
from scipy import signal
from scipy import integrate

## Radio

In [2]:
class RtlSdr:
    def __init__(self, mod = 'ook'):
        self.fs = 44100  # sampling rate
        self.mod = mod
        np.random.seed(seed=1)

    def read_samples_ook(self, n):
        fs = self.fs  # sampling rate
        baud = 300  # symbol rate
        Nbits = 10  # number of bits
        Ns = fs // baud
        N = Nbits * Ns
        f0 = 1800
        bits = randn(Nbits, 1) > 0
        M = np.tile(bits,(1, Ns))
        t = r_[0.0:N] / fs
        OOK = M.ravel() * sin(2 * pi * f0 * t)
    
        return 100.0 * OOK[:n] + 0.001 * randn(n, 1)       # very lazy way to do this, fix!

    def read_samples(self, n):
        if self.mod == 'ook':
            return self.read_samples_ook(n)
        else:
            return None

    def close(self):
        pass



## Compute SCD


In [3]:
class SCD:
    def __init__(self, N = 512, Np = 64, L = 1, mod = 'ook'):
        self.N = N
        self.Np = Np
        self.L = L
        self.r = RtlSdr(mod = mod)

    def readcompute(self):
        x = self.r.read_samples(self.N)
        scd = np.absolute(scd_fam(x, self.Np, self.L)) + 1e-6
        return scd


## GUI

In [4]:
import dearpygui.dearpygui as dpg
import numpy as np
dpg.create_context()

# This is called every frame
def update_scdwindow():
    dpg.delete_item(f"_plot")
    values = scd.readcompute()
    dpg.add_heat_series(values.flatten().tolist(), values.shape[0], values.shape[1], parent =f"_y_axis",  
                        scale_min=values.min(), scale_max=values.max(), 
                        format="", tag=f"_plot", 
                        #bounds_max=[values.shape[0]-0.5, values.shape[1]-0.5], bounds_min=[-0.5, -0.5], 
                        label="Series")
    
scd = SCD()
dpg.show_tool(dpg.mvTool_Metrics)

with dpg.window(label="SCD plot", width=1024, height=768):
    with dpg.group(horizontal=True):
        # dpg.add_colormap_scale(min_scale=values.min(), max_scale=values.max(), colormap=dpg.mvPlotColormap_Jet, tag=f"_legend")
        with dpg.plot(label="Spectral Correlation Density (FAM)", height=-1, width=-1, crosshairs=True, anti_aliased =False, delay_search=True):
            # dpg.bind_colormap(dpg.last_item(), dpg.mvPlotColormap_Jet)
            dpg.add_plot_axis(dpg.mvXAxis, tag=f"_x_axis", no_tick_marks=False, no_gridlines=False)
            dpg.add_plot_axis(dpg.mvYAxis, tag=f"_y_axis", no_tick_marks=False, no_gridlines=False)

dpg.create_viewport(title='Custom Title', width=800, height=600)
dpg.setup_dearpygui()
dpg.show_viewport()
#dpg.start_dearpygui()
while dpg.is_dearpygui_running():
    update_scdwindow()
    dpg.render_dearpygui_frame()
dpg.destroy_context()

KeyboardInterrupt: 